<a href="https://colab.research.google.com/github/michaelwnau/ai-academy-machine-learning-2023/blob/main/Temporal_GSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 3 - Session 1: Temporal GSP

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* The original code and data: https://github.com/Aujasvi-Moudgil/GSP-Implementation-Python
* This code was modified for the workshop to implement the full GSP algorithm and add the maximum span.

In [ ]:
import os
import codecs
import itertools
import operator

class GSP:
    '''GSP algorithm implementation'''
    def __init__(self, min_sup, path, max_span, GSP_mode='gsp'):
        self.min_sup = min_sup
        self.path = path
        self.unique_words, self.word_list, self.data = self.parse_data()  # parsing the data
        self.items = {}   # frequent item sets of current k-level
        self.num_users = len(self.data)   # number of sequences

        # ----------------------
        # Added variables
        self.max_span = max_span
        self.GSP_mode = GSP_mode  # to compare the efficiency of 'gsp', 'join_only', 'no_gsp'
        self.scan_count = 0 # Check the count of scanning DB

    def parse_data(self):
        files = os.listdir(self.path)
        files = [self.path + file for file in files]
        unique_words = {} # contains unique words in the data files
        unique_counter = 0 # ID number assigned to each unique word
        seqs = []
        word_list = []

        for filename in files:
            # read user data
            lines = []
            f = codecs.open(filename, encoding='utf-8')
            for line in f:
                lines.append(line)
            ans = {} # contains final time series dictionary
            seq = {}
            seq['file'] = filename
            s1 = []
            s2 = []
            # for each line in data file
            for X in lines:
                Y = str(X.strip()).split(';')
                time = Y[0].split(' ')[1]
                # extract time info
                x = []

                # extract useful info at a given time stamp
                for i,data in enumerate(Y):
                    if i > 0:
                        word = data.split('#')[1]

                        # check if word is unique or not
                        # if not, assign a new ID number
                        if word not in unique_words:
                            unique_counter = unique_counter + 1
                            unique_words[word] = unique_counter
                            word_list.append(word)
                        x.append(unique_words[word])
                # if x is not empty
                if x:
                    ans[time] = x
                    s1.extend(x)
                    s2.append(tuple(x))
            seq['data'] = ans
            seq['seq_individual'] = s1
            seq['seq_combined'] = s2
            seqs.append(seq)

        return unique_words, word_list, seqs


    # Check if x is a subsequence of y
    def is_subseq(self, x, y):
        it = iter(y)
        return all(c in it for c in x)


    # ----------------------------------------------
    # Added function: Check maximum span
    # ----------------------------------------------
    def within_max_span(self, x, y):
        first = x[0]    # get the first event of x
        last = x[-1]    # get the last event of x
        span = y.index(last) - y.index(first) # get the span from the first and the last event in y

        return span <= self.max_span
    # ----------------------------------------------


    # ----------------------------------------------
    # Modified for the timing constraint (max_span): Find support
    # When couting support, call the function "within_max_span()" to check the maximum span
    # Q: without the constraint of min_gap and max_gap, what do we assume about these constraints?
    #
    # * the original code has two flag modes:
    #  - if flag == 1: An element includes only an individual event <{a}, {b}, {c}, ....>
    #  - else: An element includes multiple events <{a, b}, {c}, ...>, but the code did not support this mode.
    # ----------------------------------------------
    def find_support(self, item, flag):
        count = 0
        if flag == 1:
            for i in range(self.num_users):

                if self.is_subseq(item, self.data[i]['seq_individual']):
                    # ----------------------------------------------
                    if self.within_max_span(item, self.data[i]['seq_individual']): # Check maximum span
                        count += 1
                    # ----------------------------------------------
        else:
            for i in range(self.num_users):
                if item in self.data[i]['seq_combined']:
                    count += 1

        self.scan_count +=1     # Added for checking the count to scan DB

        return count


    # ----------------------------------------------
    # main funciton: get support items
    # ----------------------------------------------
    def get_support_items(self, level):

        # -----------------------------------------------------
        # Step 1: Find frequent 1-items that meet min. threshold requirement
        print('Number of users = %d' % (self.num_users))
        for word in self.unique_words:
            l = [self.unique_words[word]]
            sup = self.find_support(l, 1)

            if sup >= self.min_sup:
                self.items[tuple(l)] = sup

        # If we need only 1-grams, we print those here and exit.
        if level == 1:
            sorted_patterns = sorted(self.items.items(), key=operator.itemgetter(1), reverse=True)
            for t in sorted_patterns:
                p = []
                for i in t[0]:
                    print(self.word_list[i - 1] + ' ', end='')
                print(t[1])
            return

        # -----------------------------------------------------
        # Step 2: We now generate permutations of size 2.
        # 1) get the keys of frequent 1-itemsets from self.item.keys()
        keys = [x[0] for x in self.items.keys()]
        # 2) make 2-sequence permutations, using the frequent 1-itemsets, and save them to 'self.perms'
        perms = itertools.permutations(keys, 2)
        self.perms = list(perms)
        # -----------------------------------------------------

        self.items = {}    # dictionary of item sets = { (key1, key2, ...): support_count, ...  }

        for k in keys:     # add the item sets, repeating the same item
            self.perms.append((k, k))

        for p in self.perms:
            sup = self.find_support(list(p), 1)    # get the support count of 2-item sets
            if sup >= self.min_sup:
                self.items[p] = sup

        level -= 2 # processing done till level 2


        # Step 3: processing for level > 2
        while level > 0:
            prev_items = dict(self.items) # make the current itemsets to k-1 itemsets
            self.items = {}

            # ---------------------------------------------------------------
            # * Modified:
            # Compare each step of GSP: Join, Prune, Scan
            # ---------------------------------------------------------------
            # 1. GSP mode: Join -> Prune -> Scan DB (count supports)
            if self.GSP_mode == 'gsp':
                for item in prev_items: # For every k-1 sequence,

                    # [1. Join step]
                    item_common = item[1:]   #  A: a seq, removing the first item from the first sequence

                    for item2 in prev_items:     # get the second sequence to join from all k-1 sequences,
                        item2_common = item2[:-1]   # B: a seq, removing the last item from the second sequence
                        if item_common == item2_common: # if A == B, combine the first sequence
                            new = item + item2[-1:]     # with the last item of the second sequence

                            # [2. Prune step] prune the k-sequences, having any infrequent (k-1)-subsequence
                            all_sub_freq = True
                            for subseq in itertools.combinations(new, len(new)-1): # For each (k-1)-subsequence from a new sequence
                                if subseq not in prev_items: # if any subseq is not in the k-1 frequent sequences,
                                    all_sub_freq = False     # it is false that all subsequences are frequent.

                            # [3. Scan DB]
                            if all_sub_freq:                  # only when all the k-1 subsequences are frequent,
                                sup = self.find_support(list(new), 1)   # scan DB to count supports

                                if sup >= self.min_sup:       # Save the frequent k-sequences
                                    self.items[new] = sup

            # ---------------------------------------------------------------
            # 2. No GSP pruning: Join -> Scan DB
            elif self.GSP_mode == 'join_only':
                for item in prev_items:

                    # [Join step]
                    item_common = item[1:]
                    for item2 in prev_items:
                        item2_common = item2[:-1]
                        if item_common == item2_common:
                            new = item + item2[-1:]

                            # [Scan DB]
                            sup = self.find_support(list(new), 1)
                            if sup >= self.min_sup:
                                self.items[new] = sup

            # ---------------------------------------------------------------
            # 3. No GSP joining & No GSP pruning
            else:
                for item in prev_items:
                    for k in self.unique_words:
                        key = list(item)
                        key.append(self.unique_words[k]) # combine a frequent (k-1)-sequence with every unique 1-item
                        key = tuple(key)
                        sup = self.find_support(list(key), 1)

                        if sup >= self.min_sup:
                            self.items[key] = sup

            level -= 1


        sorted_patterns = sorted(self.items.items(), key=operator.itemgetter(1), reverse=True)

        print("Count of scanning DB: {}, patterns: {} \n".format(self.scan_count, len(sorted_patterns)))

        for t in sorted_patterns:
            p = []
            for i in t[0]:
                print(self.word_list[i - 1] + '; ', end='')
            print(t[1])

        return

### Explore the efficiency of each step of GSP: Join, Prune, Scan

* GSP_mode = 'gsp'        # Join -> Prune -> Scan
* GSP_mode = 'join_only'  # Join -> Scan (No pruning)
* GSP_mode = 'no_gsp'     # Combination (k-1 frequent seq + every 1-item) -> Scan (No pruning & No Joining)

In [ ]:
minsup = 5 # minimum support
level = 3 # k-length of sequence
max_span = 10  # maxium span
g = GSP(minsup, './Data_GSP/', max_span, GSP_mode='gsp')
g.get_support_items(level)

Number of users = 45
Count of scanning DB: 1618, patterns: 130 

XXe siècle; XIXe siècle; XVIIIe siècle; 13
XVIIIe siècle; XVIIe siècle; XVIIIe siècle; 11
XXe siècle; XVIIIe siècle; XXe siècle; 10
XXe siècle; XVIIe siècle; XIXe siècle; 10
XVIIe siècle; XIXe siècle; XVIIIe siècle; 10
XXe siècle; XVIIe siècle; XXe siècle; 9
XXe siècle; XVIIe siècle; XVIIIe siècle; 9
XIXe siècle; XVIIe siècle; XIXe siècle; 9
XVIIIe siècle; XIXe siècle; XVIIIe siècle; 9
XXe siècle; XIXe siècle; XXe siècle; 8
XXe siècle; Finistère; XXe siècle; 8
XIXe siècle; XVIIIe siècle; XXe siècle; 8
XIXe siècle; XVIIIe siècle; XVIIe siècle; 8
XIXe siècle; Finistère; XIXe siècle; 8
Finistère; XXe siècle; XVIIIe siècle; 8
Finistère; XIXe siècle; XVIIIe siècle; 8
Finistère; XIXe siècle; XVIIe siècle; 8
Finistère; Grandes empires et guerres mondiales; XVIIe siècle; 8
Finistère; XVIIe siècle; XIXe siècle; 8
XVIIe siècle; XVIIIe siècle; XVIIe siècle; 8
XXe siècle; Grandes empires et guerres mondiales; XIXe siècle; 7
XXe siècl

## Questions
1. Explore how the support, level and max_span work
 * Fix the support and the level, change the max_span, and check number of patterns
 * Fix the support and the max_span, change the level, and check # of patterns
 * Fix the level and the max_span, change the support, # of patterns


2. How would changing the timestamp into actual time affect the code?
